## Yield Curve Fitting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

%matplotlib inline

### 1. Data Preparation & Cleaning

In [2]:
## If cleaned data is available, read cleaned data directly
## Else, read raw data and clean it
read_raw_data = True

In [3]:
if read_raw_data:
    ## Read raw data
    yield_raw = pd.read_csv('../data/yield.csv')
    yield_raw['Date'] = pd.to_datetime(yield_raw['Date'])

    ## Cleaning raw
    ## Get rid of data before 07/30/01 since older data had no 1 MO yield data
    yield_cleaned = yield_raw.loc[yield_raw['Date']>'2001-07-30'].reset_index(drop = True)
    ## Also get rid of 2020 data since it is our target
    yield_cleaned = yield_cleaned.loc[yield_cleaned['Date']<'2020-01-01'].reset_index(drop = True)
    ## If no 30 YR yield data, use the 20 YR yield data, since they are highly related 
    ## (See Abramov et. al.)
    yield_cleaned.loc[:,'30 YR'] = [i[1] if i[2] == 0 else i[2] 
                                    for i in yield_cleaned.loc[:,['20 YR', '30 YR']].itertuples()]
    ## Get rid of 2 MO yields since it was not introduced until late 2018
    yield_cleaned = yield_cleaned.drop('2 MO', axis = 1)
    ## Save for late use
    yield_cleaned.to_csv('../data/yield_cleaned.csv', index = False)
else:
    yield_cleaned = pd.read_csv('../data/yield_cleaned.csv')
    yield_cleaned['Date'] = pd.to_datetime(yield_cleaned['Date'])

In [4]:
## Examplar cleaned data
yield_cleaned.head(5)

,Date,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2001-07-31,3.67,3.54,3.47,3.53,3.79,4.06,4.57,4.86,5.07,5.61,5.51
1,2001-08-01,3.65,3.53,3.47,3.56,3.83,4.09,4.62,4.90,5.11,5.63,5.53
2,2001-08-02,3.65,3.53,3.46,3.57,3.89,4.17,4.69,4.97,5.17,5.68,5.57
3,2001-08-03,3.63,3.52,3.47,3.57,3.91,4.22,4.72,4.99,5.20,5.70,5.59
4,2001-08-06,3.62,3.52,3.47,3.56,3.88,4.17,4.71,4.99,5.19,5.70,5.59


In [5]:
## Cleaned data summary statistics
yield_cleaned.describe()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
count,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000,4606.000000
mean,1.297451,1.353608,1.466077,1.577794,1.823517,2.063769,2.534783,2.903291,3.235795,3.799103,3.923356
std,1.485886,1.508155,1.536025,1.498917,1.413028,1.341612,1.224258,1.138802,1.075487,1.119692,0.985139
min,0.000000,0.000000,0.020000,0.080000,0.160000,0.280000,0.560000,0.910000,1.370000,1.690000,1.940000
25%,0.070000,0.100000,0.160000,0.270000,0.640000,0.960000,1.570000,2.000000,2.280000,2.740000,3.010000
50%,0.900000,0.950000,1.040000,1.220000,1.480000,1.650000,2.320000,2.780000,3.060000,3.880000,4.040000
75%,1.967500,2.010000,2.130000,2.350000,2.660000,2.907500,3.430000,3.850000,4.190000,4.810000,4.770000
max,5.270000,5.190000,5.330000,5.300000,5.290000,5.260000,5.230000,5.290000,5.440000,6.050000,6.050000


In [6]:
## Get delta yields (aka change in yield rate, or shock)
dyield = yield_cleaned.iloc[:-1, :].copy()
dy = yield_cleaned.iloc[:, 1:].values
dyield.iloc[:, 1:] = (np.vstack([dy, np.zeros(dy.shape[1])]) - np.vstack([np.zeros(dy.shape[1]), dy]))[1:-1]

In [7]:
## Examplar delta yield
dyield.head(5)

,Date,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2001-07-31,-0.02,-0.01,0.00,0.03,0.04,0.03,0.05,0.04,0.04,0.02,0.02
1,2001-08-01,0.00,0.00,-0.01,0.01,0.06,0.08,0.07,0.07,0.06,0.05,0.04
2,2001-08-02,-0.02,-0.01,0.01,0.00,0.02,0.05,0.03,0.02,0.03,0.02,0.02
3,2001-08-03,-0.01,0.00,0.00,-0.01,-0.03,-0.05,-0.01,0.00,-0.01,0.00,0.00
4,2001-08-06,0.01,0.00,0.00,0.00,0.02,0.02,0.01,0.01,0.01,0.01,0.01


In [8]:
## Delta yield summary statistics
dyield.describe()

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
count,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000,4605.000000
mean,-0.000476,-0.000432,-0.000406,-0.000421,-0.000480,-0.000530,-0.000625,-0.000658,-0.000684,-0.000730,-0.000678
std,0.063741,0.046204,0.037007,0.037812,0.051412,0.055756,0.059802,0.060387,0.057272,0.054471,0.053763
min,-1.050000,-0.810000,-0.490000,-0.500000,-0.540000,-0.500000,-0.460000,-0.530000,-0.510000,-0.340000,-0.330000
25%,-0.010000,-0.010000,-0.010000,-0.010000,-0.020000,-0.030000,-0.030000,-0.040000,-0.030000,-0.030000,-0.030000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.010000,0.010000,0.010000,0.010000,0.020000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000
max,0.860000,0.760000,0.750000,0.520000,0.380000,0.370000,0.340000,0.300000,0.250000,0.260000,0.280000


In [9]:
## 12/31/2019 data to calculate shock for 2020 q1 prediction
last_date = yield_cleaned.iloc[-1, 1:].values

## Scenarios by the Fed
## baseline
baseline = np.array([1.6, 1.7, 1.8])
dbaseline = baseline - last_date[[1,6,8]]

## severly adverse
sevadv = np.array([0.1, 0.5, 0.7])
dsevadv = sevadv - last_date[[1,6,8]]

## 2. Nelson-Siegel Modeling

In [10]:
## Hyperparameter (See Abramov et. al.)
lmbda = 0.0299

## slope
def I1(T, lmbda = lmbda):
    return (1-np.exp(-lmbda*T))/(lmbda*T)

## curvature
def I2(T, lmbda = lmbda):
    return (1-np.exp(-lmbda*T))/(lmbda*T) - np.exp(-lmbda*T)

In [11]:
## level, slope, curvature matrix for training
I_train = np.array([[1, I1(3), I2(3)],
                    [1, I1(60), I2(60)],
                    [1, I1(120), I2(120)]])

## for prediction
I_pred = np.array([[1, I1(t), I2(t)] for t in [1, 3, 6, 12, 24, 36, 60, 84, 120, 240, 360]])

In [12]:
## train and predict baseline scenario
dbetas_baseline = np.linalg.inv(I_train).dot(dbaseline.T)
pred_baseline_NS = I_pred.dot(dbetas_baseline) + last_date

## train and predict severly adverse scenario
dbetas_sevadv = np.linalg.inv(I_train).dot(dsevadv.T)
pred_sevadv_NS = I_pred.dot(dbetas_sevadv) + last_date

In [13]:
pred_baseline_NS

array([1.5230875538830104, 1.6, 1.6584020333067235, 1.6590786719634811,
       1.6519165944693746, 1.6786852609697416, 1.7, 1.7841917707949355,
       1.8, 1.9922224810306481, 2.077374215563122], dtype=object)

In [14]:
pred_sevadv_NS

array([0.004083863752749162, 0.10000000000000009, 0.18530175911504254,
       0.234510408296434, 0.3067143596494424, 0.3944303054068028,
       0.49999999999999956, 0.6369084906887896, 0.6999999999999997,
       0.9523777039750234, 1.0576743117137615], dtype=object)

## 3. PCA

In [15]:
## Further set up
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [16]:
## Matrix to perform PCA on
X = dyield.iloc[:, 1:].values.T

## Normalize before running PCA
## Cache the mean and sd first
X_mean = np.mean(X, axis = 1)
X_sd = np.std(X, axis = 1)

## Actual normalization
X_scaled = preprocessing.scale(X, axis = 1)

In [17]:
## Perform SVD on the matrix
U, Sig, Vt = np.linalg.svd(X_scaled)

In [18]:
## array to get delta PC
dbaseline_arr = np.zeros(len(last_date))
dbaseline_arr[1] = dbaseline[0]
dbaseline_arr[6] = dbaseline[1]
dbaseline_arr[8] = dbaseline[2]

dsevadv_arr = np.zeros(len(last_date))
dsevadv_arr[1] = dsevadv[0]
dsevadv_arr[6] = dsevadv[1]
dsevadv_arr[8] = dsevadv[2]

## standarize with X's parameters
dbaseline_arr = (dbaseline_arr - X_mean)/X_sd
dsevadv_arr = (dsevadv_arr - X_mean)/X_sd

## calculate delta PC
dpc_baseline = dbaseline_arr.dot(U[:, :3])
dpc_sevadv = dsevadv_arr.dot(U[:, :3])

## calculate delta y by PCA
dy_baseline = dpc_baseline.dot(U[:, :3].T) * X_sd + X_mean
dy_sevadv = dpc_sevadv.dot(U[:, :3].T) * X_sd + X_mean

## add delta y to 12/31/2019 data to get prediction
pred_baseline_PCA = dy_baseline + last_date
pred_sevadv_PCA = dy_sevadv + last_date

In [19]:
pred_baseline_PCA

array([1.5012548089840898, 1.5690616886180833, 1.6130383239303352,
       1.5972689910177478, 1.5756222181755088, 1.6106611299482587,
       1.6730458441538492, 1.8089100275534125, 1.8973318545717504,
       2.2259364865328664, 2.366181892301054], dtype=object)

In [20]:
pred_sevadv_PCA

array([0.7123992127546557, 1.076160309543409, 1.3326838050923355,
       1.3741284010426962, 1.3882523751580051, 1.3815799153192805,
       1.3759178172906672, 1.4787154751089147, 1.5613964726224845,
       1.8829848073105957, 2.0231629288357844], dtype=object)

## 4. ANN

In [21]:
%load_ext tensorboard
import tensorflow as tf
import tensorflow.keras.backend as kb

In [22]:
## get training data
dyield_ann = dyield[['Date', '3 MO', '5 YR', '10 YR']]
dyield_ann.columns = ['Date', 'd3 MO', 'd5 YR', 'd10 YR']
X_ann = yield_cleaned.merge(dyield_ann, on = 'Date').iloc[:, 1:].values
Y_ann = dyield.iloc[:, 1:].values
## standarize training data
X_ann_mean = np.mean(X_ann, axis = 0)
X_ann_sd = np.std(X_ann, axis = 0)
X_train = preprocessing.scale(X_ann, axis = 0)
Y_train = Y_ann
X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)
## prediction target
X_pred_baseline = np.hstack([last_date, dbaseline])
X_pred_baseline = (X_pred_baseline - X_ann_mean)/X_ann_sd
X_pred_sevadv = np.hstack([last_date, dsevadv])
X_pred_sevadv = (X_pred_sevadv - X_ann_mean)/X_ann_sd

In [23]:
## the custom loss specified in Abramov et. al.
def custom_loss(y_true, y_pred):
    return kb.sum(kb.square(y_true - y_pred))

## Our neural net
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(14, activation = 'tanh'),
    tf.keras.layers.Dense(16, activation = 'tanh'),
    tf.keras.layers.Dense( 3, activation = 'tanh'),
    tf.keras.layers.Dense(16, activation = 'tanh'),
    tf.keras.layers.Dense(11, activation = 'tanh')
])

## Compile
model.compile(optimizer='adam',
              loss=custom_loss,
              metrics=[custom_loss])

## Specify log storage location
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
## Cross-val run to find optimal epochs
## Since the logs have been stored, we don't need to rerun this chunk. Check the tensor board below.
#model.fit(X_train, Y_train, validation_split = 0.8, epochs=1000, callbacks=[tensorboard_callback])

In [25]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 129948), started 2:36:46 ago. (Use '!kill 129948' to kill it.)

In [26]:
## Actual training run
model.fit(X_train, Y_train, epochs=500)

Train on 4605 samples
Epoch 1/500
4605/4605 [==============================] - 1s 193us/sample - loss: 4.6838 - custom_loss: 4.6812
Epoch 2/500
4605/4605 [==============================] - 0s 66us/sample - loss: 0.5775 - custom_loss: 0.5773
Epoch 3/500
4605/4605 [==============================] - 0s 69us/sample - loss: 0.4565 - custom_loss: 0.4563
Epoch 4/500
4605/4605 [==============================] - 0s 67us/sample - loss: 0.4217 - custom_loss: 0.4215
Epoch 5/500
4605/4605 [==============================] - 0s 70us/sample - loss: 0.3953 - custom_loss: 0.3952
Epoch 6/500
4605/4605 [==============================] - 0s 80us/sample - loss: 0.3609 - custom_loss: 0.3608
Epoch 7/500
4605/4605 [==============================] - 0s 71us/sample - loss: 0.3094 - custom_loss: 0.3092
Epoch 8/500
4605/4605 [==============================] - 0s 59us/sample - loss: 0.2649 - custom_loss: 0.2648
Epoch 9/500
4605/4605 [==============================] - 0s 59us/sample - loss: 0.2389 - custom_loss: 0.2

4605/4605 [==============================] - 0s 59us/sample - loss: 0.1497 - custom_loss: 0.1497
Epoch 76/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1480 - custom_loss: 0.1479
Epoch 77/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1469 - custom_loss: 0.1469
Epoch 78/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1473 - custom_loss: 0.1473
Epoch 79/500
4605/4605 [==============================] - 0s 74us/sample - loss: 0.1479 - custom_loss: 0.1479
Epoch 80/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1461 - custom_loss: 0.1461
Epoch 81/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1483 - custom_loss: 0.1483
Epoch 82/500
4605/4605 [==============================] - 0s 70us/sample - loss: 0.1470 - custom_loss: 0.1470
Epoch 83/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1459 - custom_loss: 0.1459
Epoch 84/500
4605/4605 

4605/4605 [==============================] - 0s 79us/sample - loss: 0.1420 - custom_loss: 0.1420
Epoch 150/500
4605/4605 [==============================] - 0s 77us/sample - loss: 0.1428 - custom_loss: 0.1428
Epoch 151/500
4605/4605 [==============================] - 0s 79us/sample - loss: 0.1456 - custom_loss: 0.1455
Epoch 152/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1424 - custom_loss: 0.1424
Epoch 153/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1421 - custom_loss: 0.1421
Epoch 154/500
4605/4605 [==============================] - 0s 62us/sample - loss: 0.1456 - custom_loss: 0.1456
Epoch 155/500
4605/4605 [==============================] - 0s 61us/sample - loss: 0.1428 - custom_loss: 0.1428
Epoch 156/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1429 - custom_loss: 0.1428
Epoch 157/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1440 - custom_loss: 0.1443
Epoch 158/500
4

Epoch 223/500
4605/4605 [==============================] - 0s 94us/sample - loss: 0.1413 - custom_loss: 0.1413
Epoch 224/500
4605/4605 [==============================] - 0s 72us/sample - loss: 0.1414 - custom_loss: 0.1414
Epoch 225/500
4605/4605 [==============================] - 0s 79us/sample - loss: 0.1424 - custom_loss: 0.1424
Epoch 226/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1417 - custom_loss: 0.1416
Epoch 227/500
4605/4605 [==============================] - 0s 72us/sample - loss: 0.1406 - custom_loss: 0.1405
Epoch 228/500
4605/4605 [==============================] - 0s 77us/sample - loss: 0.1413 - custom_loss: 0.1413
Epoch 229/500
4605/4605 [==============================] - 0s 79us/sample - loss: 0.1408 - custom_loss: 0.1407
Epoch 230/500
4605/4605 [==============================] - 0s 69us/sample - loss: 0.1418 - custom_loss: 0.1417
Epoch 231/500
4605/4605 [==============================] - 0s 84us/sample - loss: 0.1426 - custom_loss: 0.1426
E

4605/4605 [==============================] - 1s 109us/sample - loss: 0.1397 - custom_loss: 0.1396
Epoch 297/500
4605/4605 [==============================] - 1s 116us/sample - loss: 0.1400 - custom_loss: 0.1400
Epoch 298/500
4605/4605 [==============================] - 0s 103us/sample - loss: 0.1403 - custom_loss: 0.1404
Epoch 299/500
4605/4605 [==============================] - 1s 121us/sample - loss: 0.1413 - custom_loss: 0.1412
Epoch 300/500
4605/4605 [==============================] - 0s 79us/sample - loss: 0.1406 - custom_loss: 0.1407
Epoch 301/500
4605/4605 [==============================] - 0s 65us/sample - loss: 0.1398 - custom_loss: 0.1397
Epoch 302/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1411 - custom_loss: 0.1413
Epoch 303/500
4605/4605 [==============================] - 0s 69us/sample - loss: 0.1403 - custom_loss: 0.1403
Epoch 304/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1394 - custom_loss: 0.1394
Epoch 305/5

4605/4605 [==============================] - 0s 60us/sample - loss: 0.1395 - custom_loss: 0.1395
Epoch 370/500
4605/4605 [==============================] - 0s 64us/sample - loss: 0.1400 - custom_loss: 0.1400
Epoch 371/500
4605/4605 [==============================] - 0s 69us/sample - loss: 0.1383 - custom_loss: 0.1383
Epoch 372/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1393 - custom_loss: 0.1393
Epoch 373/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1386 - custom_loss: 0.1386
Epoch 374/500
4605/4605 [==============================] - 0s 65us/sample - loss: 0.1382 - custom_loss: 0.1382
Epoch 375/500
4605/4605 [==============================] - 0s 69us/sample - loss: 0.1407 - custom_loss: 0.1407
Epoch 376/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1398 - custom_loss: 0.1398
Epoch 377/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1386 - custom_loss: 0.1387
Epoch 378/500
4

Epoch 443/500
4605/4605 [==============================] - 0s 62us/sample - loss: 0.1377 - custom_loss: 0.1377
Epoch 444/500
4605/4605 [==============================] - 0s 62us/sample - loss: 0.1389 - custom_loss: 0.1390
Epoch 445/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1382 - custom_loss: 0.1382
Epoch 446/500
4605/4605 [==============================] - 0s 66us/sample - loss: 0.1392 - custom_loss: 0.1393
Epoch 447/500
4605/4605 [==============================] - 0s 60us/sample - loss: 0.1380 - custom_loss: 0.1380
Epoch 448/500
4605/4605 [==============================] - 0s 59us/sample - loss: 0.1383 - custom_loss: 0.1383
Epoch 449/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1379 - custom_loss: 0.1379
Epoch 450/500
4605/4605 [==============================] - 0s 65us/sample - loss: 0.1386 - custom_loss: 0.1389
Epoch 451/500
4605/4605 [==============================] - 0s 63us/sample - loss: 0.1379 - custom_loss: 0.1379
E

In [27]:
## predict
pred_baseline_ANN = model.predict(X_pred_baseline.reshape(1,-1).astype(np.float32)) + last_date
pred_sevadv_ANN = model.predict(X_pred_sevadv.reshape(1,-1).astype(np.float32)) + last_date

In [28]:
pred_baseline_ANN

array([[1.4845226151123643, 1.5520888659637422, 1.6203785784542561,
        1.6252276228368283, 1.6272281277179719, 1.646800060644746,
        1.6770216939598321, 1.7682680915296078, 1.821623137295246,
        2.1314151510596275, 2.2670745024085046]], dtype=object)

In [29]:
pred_sevadv_ANN

array([[0.9206496286392212, 1.0001267075538636, 1.201195901632309,
        1.1905653738975526, 1.134389238357544, 1.168327773809433,
        1.2499803328514099, 1.4363282489776612, 1.5932940971851348,
        1.9937860667705536, 2.156579419374466]], dtype=object)